In [7]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))
import numpy as np
import torch.nn as nn
from Agents.Utilities.SequentialNetwork import SequentialNetwork
from Agents.Utilities.AgentTransformations import get_continuous_agent, get_asynchronous_agent
from Agents.Utilities.ContinuousAgentMakers.ContinuousAgentMaker import ContinuousAgentMaker
from Environments.SimpleControlProblem.SimpleControlProblem import SimpleControlProblem
from Environments.SimpleControlProblem.SimpleControlProblemVisualizer import SimpleControlProblemVisualizer
import Solvers.OneAgentSolver as solver
from Agents.A2C import A2C
%load_ext autoreload
%autoreload 2

env = SimpleControlProblem()
action_n = 3
action_values = np.linspace(env.action_min, env.action_max, action_n).reshape(action_n, 1)

A2C = ContinuousAgentMaker(A2C)
pi_model = SequentialNetwork([env.state_dim, 128, 1])
v_model = SequentialNetwork([env.state_dim, 128, 1])
agent = A2C(env.action_min, env.action_max, pi_model, v_model, action_values=action_values,
             gamma=1, pi_model_lr=1e-3, v_model_lr=1e-3, entropy_threshold=0.01, action_space_type='discrete')

agent = get_asynchronous_agent(agent, inner_agent_n=10)

visualizer = SimpleControlProblemVisualizer(waiting_for_show=10)
solver.go_asynchronously(env, agent, episode_n=101, show=visualizer.show, session_len=2500, session_n=10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


RuntimeError: The size of tensor a (410) must match the size of tensor b (82) at non-singleton dimension 1